## svm.SVC from SAS® Viya® on Breast Cancer with Grid Search

### Source
This example is adapted from [SVM Hyperparameter Tuning using GridSearchCV](https://www.geeksforgeeks.org/svm-hyperparameter-tuning-using-gridsearchcv-ml/) by user tyagikartik4282.

### Data Preparation
#### About the data set
30 features describing cell nuclei were calculated from 569 digitized images of fine needle aspirations of a breast mass. Along with an identifier, each image is classified as malignant (M) or benign (B).

In addition to the parameters discovered by training a model, algorithms rely on hyperparameters to control the training process and the resulting parameters. The reasons for choosing particular values may involve a priori knowledge, intuition, or guessing, but can have a significant impact on the quality of the generated model for whatever reason they were selected.  A more methodical approach is to use a grid search algorithm that examines the space of parameter combinations to help identify the best performing hyperparameters.  This example uses `GridSearchCV` from scikit-learn to show one mechanism for doing this where the grid search is supplemented by cross-validation for model evaluation.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sasviya.ml.svm import SVC

#### Importing the data

In [ ]:
from sklearn.datasets import load_breast_cancer
dataset = load_breast_cancer()
dataset.keys()

To analyze the data, we need to extract the features and target into dataframes.  We will also examine the information about the features and show the general format of the observations.  One important thing to note is that there are no missing values for any variables.

In [ ]:
df_feat = pd.DataFrame(dataset['data'], columns = dataset['feature_names'])
df_target = pd.DataFrame(dataset['target'], columns =['Cancer'])
df_feat.info()
df_feat.head()

#### Creating training and test data
We split the original data by putting 70% into the training set and 30% into the test set. This split could be considered a hyperparameter, since it impacts the data being used for training and testing, but we will not be including it our examination.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_feat, np.ravel(df_target),
                                                    test_size = 0.30, random_state = 101)

### Training a Support Vector Classifier Model Without Tuning
Before trying hypertuning, we shall see what happens with all the defaults for a Support Vector Classifier (SVC) model.

For details about using the `SVC` class of the `sasviya` package, see the [SVC documentation](https://documentation.sas.com/?cdcId=workbenchcdc&cdcVersion=default&docsetId=explore&docsetTarget=p1udx0532v47xfn1l3ix3scjh8uj.htm).

In [ ]:
model = SVC()
model.fit(X_train, y_train)

# results analysis
y_pred = model.predict(X_test)
print("Classification report:\n", classification_report(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

### Training a Support Vector Classifier Model With Hyperparameter Tuning
Although there are multiple parameters that could be tuned for `SVC()`, we will only examine two of them: `C` and `coef0` due to the amount of time a full search would take.  Since the search space of both is continuous, we will discretize it into a handful of values of interest.  `.fit()` for a model defined with `GridSearchCV` runs the grid search first, using cross-validation to select the best option combination, and then creates a model using those options.

In [ ]:
from sklearn.model_selection import GridSearchCV
# Define the parameters to exanine and tune
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'coef0': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
grid.fit(X_train, y_train)

#### Viewing the hyperparameter values selected
Afterwards, we can list out the selected option combination and generate a model call with the parameters to use directly.

In [ ]:
print('Best parameters:', grid.best_params_)
print('Model call:', grid.best_estimator_)

#### Examining the results
Since `grid` is the model with the selected option combination, it can be interacted with like any other model object.

In [ ]:
grid_pred = grid.predict(X_test)

# print classification report
print("Classification report:\n", classification_report(y_test, y_pred))